In [1]:
import nltk
nltk.download('gutenberg')

from nltk.corpus import gutenberg

#load the data set
data = gutenberg.raw('shakespeare-hamlet.txt')

#save data in file
with open('hemlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/ravichoudhary/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


# Data preprocessing

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


In [3]:
#load the dataset
with open("hemlet.txt",'r') as file:
    text=file.read().lower()


In [4]:
#tokenize the text-creating indexes for words
tokenizer = Tokenizer()

tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words


4818

In [5]:
input_sequence=[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequence.append(n_gram_sequence)


In [6]:
max_sequence_length = max([len(x) for x in input_sequence])
max_sequence_length

14

In [7]:
input_sequence = np.array(pad_sequences(input_sequence,max_sequence_length,padding='pre'))
input_sequence

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [8]:
#load predictors and labels
x,y = input_sequence[:,:-1],input_sequence[:,-1]

In [9]:
import tensorflow as tf
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
#split the dataset in the training and testing split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [11]:
#model using LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

model = Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_length-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

#complie the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2025-09-17 13:18:38.540055: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-17 13:18:38.540490: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-17 13:18:38.540510: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-17 13:18:38.540872: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-17 13:18:38.541425: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
#model using GRU RNN

model1 = Sequential()
model1.add(Embedding(total_words,100,input_length=max_sequence_length-1))
model1.add(GRU(150,return_sequences=True))
model1.add(Dropout(0.2))
model1.add(GRU(100))
model1.add(Dense(total_words,activation='softmax'))

#complie the model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#train out LSTM RNN model
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)

Epoch 1/50


2025-09-17 13:19:11.049140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


644/644 [==============================] - 25s 34ms/step - loss: 6.8952 - accuracy: 0.0331 - val_loss: 6.7930 - val_accuracy: 0.0344
Epoch 2/50
644/644 [==============================] - 18s 28ms/step - loss: 6.4518 - accuracy: 0.0376 - val_loss: 6.8933 - val_accuracy: 0.0422
Epoch 3/50
644/644 [==============================] - 17s 26ms/step - loss: 6.3168 - accuracy: 0.0452 - val_loss: 6.9453 - val_accuracy: 0.0523
Epoch 4/50
644/644 [==============================] - 17s 26ms/step - loss: 6.1803 - accuracy: 0.0510 - val_loss: 6.9663 - val_accuracy: 0.0505
Epoch 5/50
644/644 [==============================] - 17s 26ms/step - loss: 6.0410 - accuracy: 0.0549 - val_loss: 7.0449 - val_accuracy: 0.0536
Epoch 6/50
644/644 [==============================] - 17s 26ms/step - loss: 5.9030 - accuracy: 0.0605 - val_loss: 7.1021 - val_accuracy: 0.0595
Epoch 7/50
644/644 [==============================] - 17s 26ms/step - loss: 5.7735 - accuracy: 0.0686 - val_loss: 7.1519 - val_accuracy: 0.0624
Epo

In [17]:
#train our GRU RNN model
history1=model1.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)

Epoch 1/50
644/644 [==============================] - 24s 32ms/step - loss: 7.0277 - accuracy: 0.0286 - val_loss: 6.9007 - val_accuracy: 0.0344
Epoch 2/50
644/644 [==============================] - 18s 27ms/step - loss: 6.4870 - accuracy: 0.0400 - val_loss: 6.9027 - val_accuracy: 0.0513
Epoch 3/50
644/644 [==============================] - 27s 42ms/step - loss: 6.2642 - accuracy: 0.0518 - val_loss: 6.9406 - val_accuracy: 0.0560
Epoch 4/50
644/644 [==============================] - 21s 33ms/step - loss: 6.0424 - accuracy: 0.0675 - val_loss: 6.9233 - val_accuracy: 0.0680
Epoch 5/50
644/644 [==============================] - 22s 33ms/step - loss: 5.7749 - accuracy: 0.0846 - val_loss: 6.9247 - val_accuracy: 0.0707
Epoch 6/50
644/644 [==============================] - 18s 28ms/step - loss: 5.5194 - accuracy: 0.0924 - val_loss: 7.0320 - val_accuracy: 0.0742
Epoch 7/50
644/644 [==============================] - 18s 28ms/step - loss: 5.2708 - accuracy: 0.1040 - val_loss: 7.1007 - val_accuracy:

In [18]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word ,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [19]:
input_text = "My Lord, I came to see your Fathers"
print(f"input text : {input_text}")
max_sequence_len = model1.input_shape[1]+1
next_word = predict_next_word(model1,tokenizer,input_text,max_sequence_len)
print(f"predicted next word : {next_word}")



input text : My Lord, I came to see your Fathers
1/1 [==============================] - 2s 2s/step
predicted next word : death


In [20]:
#save our model
model1.save('next_word_LSTM.h5')

#save out tokenizer
import pickle
with open('tokenizer.pkl','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

/Users/ravichoudhary/Desktop/genAI_projects/myenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
